In [2]:
import tensorflow as tf
import numpy as np

C:\Users\80LM0141IH\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k * max_pool2_k))* (input_width//(max_pool1_k * max_pool2_k)) * n_conv2 
# Keeping strides 1, the above are simple!


## wc1
![Image](media/cnn.png)
## wh1
![Image](media/cnn2.png)

In [5]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo"  : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo"  : tf.Variable(tf.random_normal([n_out])),
}

In [6]:
def conv(x,weights,bias, strides = 1):
    out = tf.nn.conv2d(x,weights,padding= "SAME",strides = [1,strides,strides,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    pooled = tf.nn.max_pool(x,padding ="SAME", ksize= [1, k, k, 1], strides = [1, k, k, 1])
    # n, height, width and finally channels
    return pooled

In [7]:
# passing it in the convolution layer 
def cnn(x,weights,biases, keep_prob):
    x = tf.reshape(x,shape= [-1, input_height,input_width, input_channels]) # -1 will make it to infer itself!
    conv1 = conv(x,weights["wc1"], biases["bc1"], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool,weights["wc2"], biases["bc2"], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape= [1,input_size_to_hidden]) # pass it through dense layer
    hidden_layer_output_before_activation = tf.add(tf.matmul(hidden_input,weights["wh1"]),biases["bh1"])
    hidden_output_before_dropout = tf.nn.relu(hidden_layer_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    # dont apply any activation for output
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [13]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases)


In [9]:
# 2 placeholders have been defined, now we can find the cost
print(pred,y)
cost= tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits = pred,
        labels = y
    ))



Tensor("Add_1:0", shape=(1, 10), dtype=float32) Tensor("Placeholder_1:0", shape=(?, 10), dtype=int32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
# batch_size = int(100)
# for i in range(25):
#     num_batches = int(mnist.train.num_examples/batch_size)
#     total_cost = 0
#     for j in range(num_batches):
#         batch_x, batch_y = mnist.train.next_batch(batch_size)
#         c,_ = sess.run([cost, optimize], feed_dict ={x:batch_x,y:batch_y})
#         total_cost+=c
#     print(total_cost)
    
batch_size = 100
c=1000
for i in range(25):
    num_batches =int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost,optimize], feed_dict={x:mnist.train.images[j:j+batch_size],y:mnist.train.labels[j:j+batch_size], keep_prob=0.8})
        total_cost+=c
    print(total_cost)

InvalidArgumentError: Input to reshape is a tensor with 313600 values, but the requested shape has 3136
	 [[{{node Reshape_1}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](MaxPool_1, Reshape_1/shape)]]

Caused by op 'Reshape_1', defined at:
  File "C:\Users\80LM0141IH\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\80LM0141IH\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\80LM0141IH\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\80LM0141IH\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\80LM0141IH\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\80LM0141IH\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177,

In [35]:
# find the highest and find the error accordingly
predictions = tf.argmax(pred,1) # index where there is maximum
correct_labels = tf.argmax(y,1)
correct_predictions = tf.equal(predictions,correct_labels)
# Strength of predictions
predictions_eval ,labels = sess.run([predictions,correct_labels],feed_dict = { x:mnist.test.images, y:fmnist.test.labels, keep_prob:1})
correct_pred = sess.run(correct_predictions,feed_dict = {x:mnist.train.images, y:mnist.train.labels})
print(correct_pred.sum())
correct_pred_test = sess.run(correct_predictions,feed_dict = {x:mnist.test.images, y:mnist.test.labels})
print('Test set predictions : ',correct_pred_test.sum())


NameError: name 'fmnist' is not defined